**~ DARKNET YOLOv4 ~**

Before starting: Runtime > Change runtime type > Hardware Acceleration > GPU

In [1]:
#[1]*10**10

In [2]:
from IPython.display import clear_output

In [3]:
# This cell imports the drive library and mounts your Google Drive as a VM local drive. You can access to your Drive files 
# using this path "/content/gdrive/My Drive/"

from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
# INSTALL TREE
# !sudo apt-get install tree
# !tree -d
# !tree gdrive/MyDrive/

In [5]:
# CHECK CUDA VERSION
# !/usr/local/cuda/bin/nvcc --version

In [ ]:
# INCLUDE CUDA
# %cd /content/gdrive/My\ Drive/MyDrive-sync/darknet/cuDNN
# %cp  cudnn-10.1-linux-x64-v8.0.5.39.solitairetheme8 cudnn-10.1-linux-x64-v8.0.5.39.tgz
!tar -xzvf /content/gdrive/MyDrive/cuDNN/cudnn-10.1-linux-x64-v8.0.5.39.tgz -C /usr/local/
#!chmod a+r /usr/local/cuda/include/cudnn.h
!chmod a+r /usr/local/cuda-10.1/include/cudnn.h

In [7]:
# CLONE DARKNET REPOSITORY
# !git clone https://github.com/kriyeng/darknet/ for v3 or https://github.com/AlexeyAB/darknet for v4
# %cd darknet

In [8]:
# CHECK CUDA PATH FILES THEN MAKE EDITS TO Makefile.txt by changing cuda -> cuda-10.1 and lib -> lib64 (also make sure GPU=1, OPENCV=1 in txt file!)
# !ls /usr/local/cuda-10.1

In [9]:
# MAKE DARKNET
## !git checkout feature/google-colab
# !make

In [10]:
# COPY NEWLY COMPILED DARKNET REPO TO LOCAL GOOGLE-DRIVE
# %cd ..
# !cp -r darknet /content/gdrive/MyDrive/
# !ls /content/gdrive/MyDrive/

In [11]:
# COPY COMPILED DARKNET REPO FROM LOCAL GOOGLE-DRIVE TO VM DRIVE
!cp -r /content/gdrive/MyDrive/darknet /content
#!cp /content/gdrive/MyDrive/yolov4.weights /content/darknet

In [12]:
# GET YOLOv4 WEIGHTS
# %cd /content/gdrive/MyDrive/darknet/
# !wget https://pjreddie.com/media/files/yolov3.weights for v3 or https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights for v4

In [13]:
# GET YOLOv4 CONVOLUTIONAL WEIGHTS (Convolutional layer: adds pre-trained weights for convolutional layer, for faster converging of the training model)
# !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

Checking if this all works:

In [18]:
# CREATE SIMPLE CV2 COMMANDS

def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
  
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
      
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
# YOLO DETECTION
#%cd darknet
!chmod +x darknet
!./darknet detect cfg/yolov4.cfg yolov4.weights cats.jpg -dont-show
imShow('predictions.jpg')

In [ ]:
# STOPS THE AUTOMATIC RUNTIME HERE
assert False

**Configuration Files for a custom dataset (local folder yolov4)**

1. Image Datasets: after converting annotations, rename, zip and copy over the img datasets

In [ ]:
# COPY OBJ- & TEST- ZIP FILES TO VM DARKNET
!cp /content/gdrive/MyDrive/yolov4/obj.zip /content/darknet
!cp /content/gdrive/MyDrive/yolov4/test.zip /content/darknet
!unzip obj.zip -d data/
!unzip test.zip -d data/

2. .cfg file: copy from cfg folder (or edit existing file) in local env.

In [ ]:
# GENERATE A NEW cfg FILE FOR EDITS
# !cp /content/gdrive/MyDrive/darknet/cfg/yolov4-custom.cfg /content/gdrive/MyDrive/yolov4/cfg/yolov4-obj.cfg #This copies it and edits the name!
# Open up the yolov4-obj.cfg file and make changes to parameters for the custom object detector!

In [37]:
# COPY THE EDITED .cfg FILE BACK INTO VM
!cp /content/gdrive/MyDrive/yolov4/cfg/yolov4-obj.cfg /content/darknet/cfg

3. obj.names & obj.data files: create (or edit existing file) in local env.

In [32]:
# EDIT (OR MAKE) obj.names & obj.data FILES, THEN COPY INTO VM
!cp /content/gdrive/MyDrive/yolov4/obj.names /content/darknet/data
!cp /content/gdrive/MyDrive/yolov4/obj.data /content/darknet/data

4. train.txt & test.txt files: create (or edit existing file) in loval env.

In [ ]:
# COPY PYTHON SCRIPTS INTO VM FOR GENERATING THE TXT FILES, THEN RUN
!python /content/gdrive/MyDrive/yolov4/generate_test.py
!python /content/gdrive/MyDrive/yolov4/generate_train.py
!cp /content/gdrive/MyDrive/yolov4/test.txt /content/darknet/
!cp /content/gdrive/MyDrive/yolov4/train.py /content/darknet/
!ls darknet

**Run training!**

In [ ]:
#%%capture <-uncomment if memory errors occur!

!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

# NOTE:
# ・for weights, can use: yolov4.conv.137 or yolov4.weights !!
# ・include "-map" when using validation datasets! (shows iterations, etc)

In [ ]:
# TO CONTINUE INTERRUPTED TRAINING WITH BACKUP WEIGHTS:
!./darknet detector train data/obj.data /content/gdrive/MyDrive/yolov4-obj.cfg /content/gdrive/MyDrive/darknet/backup/yolov4-obj_last.weights -dont_show -map

In [ ]:
# TO COPY OVER PREVIOUS CONFIG FILES:
!cp 

Check results of training:

In [ ]:
# TO CHECK THE PROGRESS OF THE TRAINING:
imShow('chart.png')

In [ ]:
# TO CHECK THE Mean Average Precision (mAP) OF THE MODEL
!./darknet detector map /content/gdrive/MyDrive/yolov4/obj.data /content/gdrive/MyDrive/yolov4/cfg/yolov4-obj.cfg /content/gdrive/MyDrive/darknet/backup/yolov4-obj_last.weights

# NOTE:
# ・it is ideal to run this on multiple of the 'last saved weights' to compare and find the weights with the highest mAP since that is the most accurate one!

In [ ]:
# YOLO DETECTION:
%cd cfg
!sed -i 's/batch=64/batch=1/' cfg/yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' cfg/yolov4-obj.cfg
%cd ..

!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /content/gdrive/MyDrive/yolov4/backup/yolov4-obj_last.weights /content/gdrive/MyDrive/cats.jpg -thresh 0.3
imShow('predictions.jpg')

# NOTE:
# ・the thresh flag sets the accuracy that detection must be in order to show it